<a href="https://colab.research.google.com/github/rjsheesh/RJ-Sheesh/blob/master/url2gd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
import threading
import requests
from tqdm.auto import tqdm

def download_part(url, start, end, part_file, pbar, retries=5):
    headers = {'Range': f'bytes={start}-{end}'}
    for attempt in range(retries):
        try:
            with requests.get(url, headers=headers, stream=True, timeout=30) as r:
                r.raise_for_status()
                with open(part_file, "wb") as f:
                    for chunk in r.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                            pbar.update(len(chunk))
            break
        except Exception as e:
            if attempt < retries - 1:
                print(f"পার্ট {part_file} ডাউনলোডে সমস্যা, আবার চেষ্টা করছি... ({attempt+1}/{retries})")
            else:
                raise Exception(f"পার্ট {part_file} ডাউনলোড ব্যর্থ: {e}")

def merge_parts(part_files, output_file):
    with open(output_file, 'wb') as outfile:
        for part_file in part_files:
            with open(part_file, 'rb') as infile:
                outfile.write(infile.read())
            os.remove(part_file)  # পার্ট ফাইলগুলো ডিলিট করি

def smart_downloader(url, output_path, num_threads=4):
    response = requests.head(url)
    if 'Content-Length' not in response.headers:
        raise Exception("সার্ভার Content-Length দেয়নি, মাল্টি-পার্ট ডাউনলোড সম্ভব না।")

    total_size = int(response.headers['Content-Length'])
    part_size = total_size // num_threads

    part_files = []
    threads = []
    pbar = tqdm(total=total_size, unit='B', unit_scale=True, desc=os.path.basename(output_path))

    for i in range(num_threads):
        start = part_size * i
        end = start + part_size - 1 if i != num_threads - 1 else total_size - 1
        part_file = f"{output_path}.part{i}"
        part_files.append(part_file)
        t = threading.Thread(target=download_part, args=(url, start, end, part_file, pbar))
        t.start()
        threads.append(t)

    for t in threads:
        t.join()

    pbar.close()

    merge_parts(part_files, output_path)
    print("\nডাউনলোড সফলভাবে সম্পূর্ণ হয়েছে!")

if __name__ == "__main__":
    url = input("ডাইরেক্ট URL দাও: ").strip()
    final_name = url.split("/")[-1]
    save_path = f"/content/drive/Shareddrives/15/Torrent/{final_name}"

    threads = input("কতটা থ্রেডে নামাতে চাও? (ডিফল্ট ৪): ").strip()
    threads = int(threads) if threads.isdigit() else 4

    smart_downloader(url, save_path, num_threads=threads)